## This note book is currently in the works

I want my model to be explainable. One of the best ways to do that is o generate attention for each generated caption. I want to be able to hover over any generated work and show a heatmap layed over the input image explaining why that word was generated. This is harder than I thought but we'll get there!

In [ ]:
# import torch
# import matplotlib.pyplot as plt
# import numpy as np
# import os
# from transformers import BlipProcessor, BlipForConditionalGeneration
# from peft import PeftModel
# from PIL import Image
# import cv2


# def get_attention_maps(model):
#     attention_maps = []

#     def hook_fn(module, input, output):
#         # output: attention weights shape (batch_size, num_heads, tgt_len, src_len)
#         attention_maps.append(output[1].detach().cpu())  # cross-attention weights

#     # Access decoder cross-attention layers
#     for layer in model.text_decoder.bert.encoder.layer:
#         # Register on the cross-attention module
#         layer.crossattention.register_forward_hook(hook_fn)

#     return attention_maps

# def visualize_attention(image, attention_map, word, patch_size=16):
#     # attention_map: (1, 577) → remove CLS
#     attn = attention_map[0, 1:]  # Remove CLS token, now shape [576]
#     print(attn)

#     # Compute patch grid
#     num_patches = attn.shape[0]
#     h = w = int(np.sqrt(num_patches))
#     if h * w != num_patches:
#         raise ValueError(f"Cannot reshape attention of size {num_patches} into square grid")

#     attn = attn.reshape(h, w).numpy()
#     attn = (attn - attn.min()) / (attn.max() - attn.min())  # normalize to [0,1]

#     # Resize attention to match image size
#     attn_resized = cv2.resize(attn, image.size)  # use OpenCV to resize heatmap

#     # Overlay heatmap on image
#     plt.imshow(image)
#     plt.imshow(attn_resized, cmap='jet', alpha=0.5)
#     plt.title(f"Attention for '{word}'")
#     plt.axis('off')
#     plt.show()

# def describe_image_with_attention(img):
#     base_model_name = "Salesforce/blip-image-captioning-base"
#     adapter_path = os.path.join('./blip-lora-finetuned/checkpoint-4770/')
#     device = "cuda" if torch.cuda.is_available() else "cpu"

#     processor = BlipProcessor.from_pretrained(base_model_name)
#     base_model = BlipForConditionalGeneration.from_pretrained(base_model_name)
#     model = PeftModel.from_pretrained(base_model, adapter_path).to(device)
#     model.eval()

#     inputs = processor(images=img, return_tensors="pt").to(device)
#     pixel_values = inputs['pixel_values']

#     # Capture attention maps
#     attention_maps = get_attention_maps(model)

#     # Generate caption
#     with torch.no_grad():
#         outputs = model.generate(
#             pixel_values=pixel_values,
#             max_length=20,
#             num_beams=5,
#             early_stopping=True,
#             output_attentions=True,
#             return_dict_in_generate=True
#         )

#     caption_ids = outputs.sequences[0]
#     caption = processor.decode(caption_ids, skip_special_tokens=True)
#     print("Caption:", caption)

#     # Visualize attention for selected words
#     words = caption.split()
#     for i, word in enumerate(words):
#         if i < len(attention_maps):  # Safety check
#             visualize_attention(img, attention_maps[i][0], word)

#     return caption


In [ ]:
# describe_image_with_attention(Image.open('./glass.jpg').convert("RGB"))

Error during conversion: ChunkedEncodingError(ProtocolError("Connection broken: InvalidChunkLength(got length b'', 0 bytes read)", InvalidChunkLength(got length b'', 0 bytes read)))


Caption: a clear glass on a white background
tensor([], size=(0, 577))


ValueError: zero-size array to reduction operation minimum which has no identity

In [36]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import cv2
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration
from peft import PeftModel

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- Load model ---
base_model_name = "Salesforce/blip-image-captioning-base"
adapter_path = "./blip-lora-finetuned/checkpoint-4770/"

processor = BlipProcessor.from_pretrained(base_model_name)
base_model = BlipForConditionalGeneration.from_pretrained(base_model_name)
model = PeftModel.from_pretrained(base_model, adapter_path).to(device)
model.eval()

# --- Load and preprocess image ---
raw_image = Image.open('./glass.jpg').convert("RGB")
inputs = processor(images=raw_image, return_tensors="pt").to(device)
pixel_values = inputs['pixel_values']

# --- Hook setup ---
activations = None
gradients = None

def forward_hook(module, input, output):
    global activations
    activations = output

def backward_hook(module, grad_input, grad_output):
    global gradients
    gradients = grad_output[0]

# Register hooks on last encoder layer's attention QKV
target_layer = model.base_model.vision_model.encoder.layers[-1].self_attn.qkv
fwd_handle = target_layer.register_forward_hook(forward_hook)
bwd_handle = target_layer.register_full_backward_hook(backward_hook)

# --- Forward with gradient ---
model.train()  # Enable gradients (important!)
outputs = model.generate(
    pixel_values=pixel_values,
    max_length=30,
    num_beams=1,
    output_scores=True,
    return_dict_in_generate=True
)

# --- Pick target token index (e.g. 3rd token) ---
token_ids = outputs.sequences[0]
token_index = 3
target_token_id = token_ids[token_index]
token_score = outputs.scores[token_index - 1][0, target_token_id]  # scores are offset by 1

# --- Backward pass ---
model.zero_grad()
token_score.backward(retain_graph=True)

# --- Compute CAM ---
weights = gradients.mean(dim=1, keepdim=True)         # (1, 1, D)
cam = (weights * activations).sum(dim=-1)             # (1, N)
cam = F.relu(cam)
cam = cam / cam.max()
cam = cam.squeeze().detach().cpu().numpy()

# Remove CLS token (if ViT), reshape
cam = cam[1:]  # BLIP ViT uses [CLS] at index 0
num_patches = int(cam.shape[0] ** 0.5)
cam = cam.reshape(num_patches, num_patches)
cam = cv2.resize(cam, raw_image.size)  # Resize to image size
heatmap = cv2.applyColorMap(np.uint8(255 * cam), cv2.COLORMAP_JET)

# --- Overlay and visualize ---
superimposed_img = heatmap * 0.4 + np.array(raw_image)

decoded_token = processor.tokenizer.decode([target_token_id])
plt.imshow(superimposed_img.astype(np.uint8))
plt.title(f"Token: '{decoded_token}'")
plt.axis('off')
plt.show()

# Clean up
fwd_handle.remove()
bwd_handle.remove()


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn